In [1]:
import pandas as pd
import numpy as np
from rank_bm25 import *
from rank_bm25 import BM25Okapi
import nltk
import string
import unidecode
import re
from string import punctuation
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import wordnet

In [2]:
news_data = pd.read_csv('NEWSAPI_data.csv')
news_data

,sno,headline,description,content,url,author,date,ticker
0,0,58 Black investors changing the venture capita...,NaN,"From left: Paul Judge, chairman of The Open Op...",https://www.businessinsider.com/black-venture-...,Vishal Persaud,2024-02-20T10:00:01Z,MMM
1,1,3M electric toothbrushes were not used in a DD...,A widely reported story that 3 million electri...,Update added below with Fortinet's statement c...,https://www.bleepingcomputer.com/news/security...,Lawrence Abrams,2024-02-08T10:29:13Z,MMM
2,2,The story of Signature Plastics,President Bob Guenser sums up the history of S...,The story of Signature Plastics\r\nPresident B...,https://kbd.news/The-story-of-Signature-Plasti...,NaN,2024-02-12T13:01:07Z,MMM
3,3,Carsifi Wireless Android Auto Adapter review: ...,It'll take a specific use case for this adapte...,If you’ve purchased or rented a new car in the...,https://www.androidpolice.com/carsifi-wireless...,Jesse Hollington,2024-03-03T16:00:14Z,MMM
4,4,The Industrial Designer behind the N95 Mask,Sara Little Turnbull used materials science to...,Sara Little Turnbull was a force in the world ...,https://www.scientificamerican.com/article/the...,"Johanna Mayer, Katie Hafner, The Lost Women of...",2024-02-23T22:00:00Z,MMM
...,...,...,...,...,...,...,...,...
2875,95,Synovus Financial Corp Has $1.11 Million Holdi...,Synovus Financial Corp increased its stake in ...,Synovus Financial Corp increased its stake in ...,https://www.etfdailynews.com/2024/03/02/synovu...,MarketBeat News,2024-03-02T16:20:49Z,WBA
2876,96,Blair William & Co. IL Reduces Stock Holdings ...,Blair William & Co. IL trimmed its position in...,Blair William &amp; Co. IL trimmed its positio...,https://www.etfdailynews.com/2024/02/14/blair-...,MarketBeat News,2024-02-14T13:56:44Z,WBA
2877,97,"Walgreens Boots Alliance, Inc. (NASDAQ:WBA) Re...","Walgreens Boots Alliance, Inc. (NASDAQ:WBA – G...","Walgreens Boots Alliance, Inc. (NASDAQ:WBA – G...",https://www.etfdailynews.com/2024/02/09/walgre...,MarketBeat News,2024-02-09T07:30:44Z,WBA
2878,98,Profund Advisors LLC Has $1.50 Million Holding...,Profund Advisors LLC raised its position in Wa...,Profund Advisors LLC raised its position in Wa...,https://www.etfdailynews.com/2024/02/26/profun...,MarketBeat News,2024-02-26T14:38:41Z,WBA


In [3]:
news_data.isnull().sum()

sno              0
headline         0
description     70
content          0
url              0
author         200
date             0
ticker           0
dtype: int64

In [4]:
yt_data = pd.read_csv('YT_data.csv')
yt_data

,title,link,Summary,published,Ticker
0,Dividend Cut After 3M Spin Off This Week? | 3M...,www.youtube.com/watch?v=rIdpV4n9pjo,In the YouTube video titled Dividend Cut After...,2024-03-30,MMM
1,3M starts New Year with ridding of pensions &a...,www.youtube.com/watch?v=qLE9oNiRQNk,"In the YouTube video titled ""3M starts New Yea...",2024-01-11,MMM
2,TATA POWER Q3 RESULTS 2024 DETAIL ANALYSIS ⚫ T...,www.youtube.com/watch?v=6DJJP9_qFng,"In the YouTube video titled ""TATA POWER Q3 Res...",2024-02-09,MMM
3,Jim Cramer looks ahead to next week&#39;s earn...,www.youtube.com/watch?v=MnipJpbWR_E,"In the ""Jim Cramer looks ahead to next week's ...",2024-01-20,MMM
4,Share Market SCAM | Exposed By Sandeep Maheshw...,www.youtube.com/watch?v=OejIq2kM04c,"In the YouTube video ""Share Market SCAM | Expo...",2024-01-29,MMM
...,...,...,...,...,...
101,"Coalition Fighting Disney DAS Changes, Disneyl...",www.youtube.com/watch?v=_khkPgv85Ao,"Disney has launched My Disney, a website that ...",2024-04-17,DIS
102,Disney to Ban Guests Who Lie About Disabilitie...,www.youtube.com/watch?v=vDdGRx6ONME,Disney's Hollywood Studios is celebrating its ...,2024-04-11,DIS
103,Disney Cracking Down on Disability Access Service,www.youtube.com/watch?v=ufIOBhBHVoU,The 35th anniversary of Disney's Hollywood Stu...,2024-04-08,DIS
104,"Guest Falls in River at Magic Kingdom, Disney ...",www.youtube.com/watch?v=OVg6sxS1aUM,Disney's federal lawsuit against Florida Gover...,2024-02-02,DIS


In [5]:
yt_data.isnull().sum()

title        0
link         0
Summary      0
published    0
Ticker       0
dtype: int64

In [6]:
news_data['text'] = news_data['headline'].fillna('') + ' ' + news_data['description'].fillna('') + ' ' + news_data['content'].fillna('')
yt_data['text'] = yt_data['title'].fillna('') + ' ' + yt_data['Summary'].fillna('')

In [7]:
news_data

,sno,headline,description,content,url,author,date,ticker,text
0,0,58 Black investors changing the venture capita...,NaN,"From left: Paul Judge, chairman of The Open Op...",https://www.businessinsider.com/black-venture-...,Vishal Persaud,2024-02-20T10:00:01Z,MMM,58 Black investors changing the venture capita...
1,1,3M electric toothbrushes were not used in a DD...,A widely reported story that 3 million electri...,Update added below with Fortinet's statement c...,https://www.bleepingcomputer.com/news/security...,Lawrence Abrams,2024-02-08T10:29:13Z,MMM,3M electric toothbrushes were not used in a DD...
2,2,The story of Signature Plastics,President Bob Guenser sums up the history of S...,The story of Signature Plastics\r\nPresident B...,https://kbd.news/The-story-of-Signature-Plasti...,NaN,2024-02-12T13:01:07Z,MMM,The story of Signature Plastics President Bob ...
3,3,Carsifi Wireless Android Auto Adapter review: ...,It'll take a specific use case for this adapte...,If you’ve purchased or rented a new car in the...,https://www.androidpolice.com/carsifi-wireless...,Jesse Hollington,2024-03-03T16:00:14Z,MMM,Carsifi Wireless Android Auto Adapter review: ...
4,4,The Industrial Designer behind the N95 Mask,Sara Little Turnbull used materials science to...,Sara Little Turnbull was a force in the world ...,https://www.scientificamerican.com/article/the...,"Johanna Mayer, Katie Hafner, The Lost Women of...",2024-02-23T22:00:00Z,MMM,The Industrial Designer behind the N95 Mask Sa...
...,...,...,...,...,...,...,...,...,...
2875,95,Synovus Financial Corp Has $1.11 Million Holdi...,Synovus Financial Corp increased its stake in ...,Synovus Financial Corp increased its stake in ...,https://www.etfdailynews.com/2024/03/02/synovu...,MarketBeat News,2024-03-02T16:20:49Z,WBA,Synovus Financial Corp Has $1.11 Million Holdi...
2876,96,Blair William & Co. IL Reduces Stock Holdings ...,Blair William & Co. IL trimmed its position in...,Blair William &amp; Co. IL trimmed its positio...,https://www.etfdailynews.com/2024/02/14/blair-...,MarketBeat News,2024-02-14T13:56:44Z,WBA,Blair William & Co. IL Reduces Stock Holdings ...
2877,97,"Walgreens Boots Alliance, Inc. (NASDAQ:WBA) Re...","Walgreens Boots Alliance, Inc. (NASDAQ:WBA – G...","Walgreens Boots Alliance, Inc. (NASDAQ:WBA – G...",https://www.etfdailynews.com/2024/02/09/walgre...,MarketBeat News,2024-02-09T07:30:44Z,WBA,"Walgreens Boots Alliance, Inc. (NASDAQ:WBA) Re..."
2878,98,Profund Advisors LLC Has $1.50 Million Holding...,Profund Advisors LLC raised its position in Wa...,Profund Advisors LLC raised its position in Wa...,https://www.etfdailynews.com/2024/02/26/profun...,MarketBeat News,2024-02-26T14:38:41Z,WBA,Profund Advisors LLC Has $1.50 Million Holding...


In [8]:
combined_data = pd.concat([
    news_data[['text', 'date', 'url', 'ticker']], yt_data[['text', 'published', 'link', 'Ticker']].rename(columns={'link': 'url', 'published': 'date', 'Ticker': 'ticker'})
], ignore_index=True)

combined_data

,text,date,url,ticker
0,58 Black investors changing the venture capita...,2024-02-20T10:00:01Z,https://www.businessinsider.com/black-venture-...,MMM
1,3M electric toothbrushes were not used in a DD...,2024-02-08T10:29:13Z,https://www.bleepingcomputer.com/news/security...,MMM
2,The story of Signature Plastics President Bob ...,2024-02-12T13:01:07Z,https://kbd.news/The-story-of-Signature-Plasti...,MMM
3,Carsifi Wireless Android Auto Adapter review: ...,2024-03-03T16:00:14Z,https://www.androidpolice.com/carsifi-wireless...,MMM
4,The Industrial Designer behind the N95 Mask Sa...,2024-02-23T22:00:00Z,https://www.scientificamerican.com/article/the...,MMM
...,...,...,...,...
2981,"Coalition Fighting Disney DAS Changes, Disneyl...",2024-04-17,www.youtube.com/watch?v=_khkPgv85Ao,DIS
2982,Disney to Ban Guests Who Lie About Disabilitie...,2024-04-11,www.youtube.com/watch?v=vDdGRx6ONME,DIS
2983,Disney Cracking Down on Disability Access Serv...,2024-04-08,www.youtube.com/watch?v=ufIOBhBHVoU,DIS
2984,"Guest Falls in River at Magic Kingdom, Disney ...",2024-02-02,www.youtube.com/watch?v=OVg6sxS1aUM,DIS


In [9]:
combined_data.isna().sum()

text      0
date      0
url       0
ticker    0
dtype: int64

In [15]:
lemmatizer = WordNetLemmatizer()

def WordNetTags(tag):
    if tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return None
    
def preprocessText(sen):
    # Lowercase
    sen = sen.lower()

    # Remove any HTML Tags
    sen = re.sub(r'<.*?>', '', sen)
    
    # Remove punctuations
    sen = [i for i in sen if i not in string.punctuation]
    sen = ''.join(sen)

    # Remove accented characters
    sen = unidecode.unidecode(sen)
    
    # Work tokenize
    sen = nltk.word_tokenize(sen)
    
    # Remove stopwords
    stop_words = set(nltk.corpus.stopwords.words('english'))
    sen = [i for i in sen if i not in stop_words]
    
    # remove blank space token
    sen = [i for i in sen if i.strip() != '' ]
    
    # Lemmatize
    sen = nltk.pos_tag(sen)
    sen = [lemmatizer.lemmatize(i, WordNetTags(tag)) if WordNetTags(tag) else lemmatizer.lemmatize(i) for i,tag in sen]
    # sen = ' '.join(sen)
    
    return sen
    


In [16]:
corpusdata = combined_data['text'].apply(preprocessText).tolist()

In [17]:
bm25 = BM25Okapi(corpusdata)

In [20]:
def get_k_results(query, k = 5, ticker = None, dt = None):
    # Preprocess Query Text
    query = preprocessText(query)

    # Get scores
    scores = bm25.get_scores(query)

    # Entries for specified company name and date
    output = combined_data

    if ticker: # For company
        output = output[output['ticker'].str.contains(ticker, case=False)]
    
    if dt: # for date range
        output = output[output['date'] >=dt[0]] & output[output['date'] <= dt[1]]

    # Give the k top entries based on scores from 'output' df

    output_index = np.argsort(scores[output.index])[-k:][::-1]

    return output.iloc[output_index]



In [21]:
input_query = "HI i was wondering how the apple stocks are working?!."

get_k_results(input_query, ticker='AAPL')

,text,date,url,ticker
303,Apple Spent More Than $10 Billion on Apple Car...,2024-02-29T01:13:55Z,https://www.macrumors.com/2024/02/28/apple-car...,AAPL
329,Will Apple create a better AI? Questions aboun...,2024-02-29T02:17:00Z,https://www.marketwatch.com/story/will-apple-c...,AAPL
327,Apple’s stock is in a ‘funk.’ Could Warren Buf...,2024-03-05T17:00:00Z,https://www.marketwatch.com/story/apples-stock...,AAPL
310,Apple buys iWork.ai domain amid Big Tech AI ra...,2024-02-15T15:10:35Z,https://readwrite.com/apple-buys-iwork-ai-doma...,AAPL
317,"Building for DOS, OS/2, and DOS on a MacBook A...",2024-02-10T11:05:33Z,https://retrocoding.net/building-for-dos-os2-a...,AAPL


In [22]:
input_query = "AREE APPLE STOCKS SINKING HUH? 1"

get_k_results(input_query, ticker='AAPL')

,text,date,url,ticker
327,Apple’s stock is in a ‘funk.’ Could Warren Buf...,2024-03-05T17:00:00Z,https://www.marketwatch.com/story/apples-stock...,AAPL
2891,Apple bull vs. bear debate: What&#39;s next fo...,2024-03-01,www.youtube.com/watch?v=dAMRdOlfJPo,AAPL
356,'Constellation' season 1 episode 5 review: Les...,2024-03-06T20:30:01Z,https://www.space.com/constellation-season-1-e...,AAPL
303,Apple Spent More Than $10 Billion on Apple Car...,2024-02-29T01:13:55Z,https://www.macrumors.com/2024/02/28/apple-car...,AAPL
381,How Will New IPads And Higher IPhone Pricing H...,2024-02-16T11:00:54Z,https://www.forbes.com/sites/greatspeculations...,AAPL
